In [2]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=6cbd57978c24049fb8b4af71831e49318bc8cfc764eeb696984efa739d0bc0f2
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


In [5]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/week3/train.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/week3/test.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 595)
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...            

In [0]:
train_data.columns

Index(['CoverageField11A', 'CoverageField11B', 'CoverageField1A',
       'CoverageField1B', 'CoverageField2A', 'CoverageField2B',
       'CoverageField3A', 'CoverageField3B', 'CoverageField4A',
       'CoverageField4B',
       ...
       'PropertyField38_N', 'PropertyField38_Y', 'GeographicField63_ ',
       'GeographicField63_N', 'GeographicField63_Y', 'GeographicField64_CA',
       'GeographicField64_IL', 'GeographicField64_NJ', 'GeographicField64_TX',
       'QuoteConversion_Flag'],
      dtype='object', length=596)

In [6]:
#Copy Train data excluding target
trainData_Copy = train_data.iloc[:, :-1].copy()
testData_Copy = test_data.copy()

X_train = trainData_Copy
X_test = testData_Copy
y_train=train_data["QuoteConversion_Flag"]
#y_test=test_data["QuoteConversion_Flag"]
print("--------------------------------")

print(X_train.shape)
print(X_test.head())
print(y_train.shape)

--------------------------------
(65000, 595)
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...                     0
2                 3  ...                     0
3                 5  ...                     0
4                12  ...                     0

[5 rows x 595 columns]
(65000,)


In [7]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [0]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_res, y_res)
test_pred=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"])
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/result1_DTC.csv")



In [36]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_res, y_res)
grid_parm=clf_random.best_params_
print(grid_parm)


{'min_samples_split': 80, 'max_depth': 1}


In [0]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf_hyper = DecisionTreeClassifier(**grid_parm)
clf_hyper.fit(X_res,y_res)
test_pred=pd.DataFrame(clf_hyper.predict(X_test),columns=["QuoteConversion_Flag"])

In [0]:
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/result2.csv")

In [42]:
#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_res, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

=== All AUC Scores ===
[0.78469852 0.61452408 0.73208191 0.67576792 0.80991657 0.95610542
 0.9496587  0.90386803 0.93827169 0.5055935 ]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7870486348266648


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
test_pred=pd.DataFrame(rfc.predict(X_test),columns=["QuoteConversion_Flag"])
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/Result_RF.csv")




In [45]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)


{'n_estimators': 110, 'min_samples_split': 50, 'max_depth': 19}


In [44]:
#contruct random forest using the best parameters
rfc_hyper= RandomForestClassifier(**grid_parm_rfc)
rfc_hyper.fit(X_train,y_train)
test_pred=pd.DataFrame(rfc_hyper.predict(X_test),columns=["QuoteConversion_Flag"])
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/Result_RF_Hyperparam.csv")


NameError: ignored

In [27]:
#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_res, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

=== All AUC Scores ===
[0.93737195 0.93173738 0.93925431 0.94202396 0.95324802 0.99950365
 0.99967377 0.99943232 0.99952405 0.99826398]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9700033392900103


In [0]:
#CONSTRUCT DEFAULT NN AND OBTAIN RESPECTIVE ACCURACY =======================
mlp = MLPClassifier()
mlp.fit(X_res, y_res)
test_pred=pd.DataFrame(mlp.predict(X_test),columns=["QuoteConversion_Flag"])
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/Result_NN.csv")


In [30]:
#HYPERPARAMETERTUNING DONE FOR NN CLASSIFIER =======================
params={'hidden_layer_sizes':range(10,100,10), 'max_iter':range(10,500,10), 'alpha':range(10,100,10)}
mlp_random = RandomizedSearchCV(mlp,params,n_iter=15)
mlp_random.fit(X_res, y_res)
grid_parm=mlp_random.best_params_
print(grid_parm)


KeyboardInterrupt: ignored

In [5]:
#Using the parameters obtained from HyperParameterTuning in the NN 
mlp = MLPClassifier(**grid_parm)
mlp.fit(X_res,y_res)
test_pred=pd.DataFrame(mlp.predict(X_test),columns=["QuoteConversion_Flag"])

NameError: ignored

In [0]:
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/ResultNnHyperParam.csv")

In [34]:
#run cross-validation on best hyperparameters, get auc score
clfCvScore = cross_val_score(clf, X_res, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clfCvScore)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clfCvScore.mean())

=== All AUC Scores ===
[0.74554418 0.74810391 0.74611301 0.75739477 0.77379598 0.81484642
 0.818449   0.81190747 0.81661549 0.82002421]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7852794420303368


In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
#CONSTRUCT DEFAULT KNN AND OBTAIN RESPECTIVE ACCURACY =====================
knn = KNeighborsClassifier()
knn.fit(X_res, y_res)
test_pred=pd.DataFrame(knn.predict(X_test),columns=["QuoteConversion_Flag"])
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/Result_KNN.csv")

In [0]:
#Hyperparameter tuning done for KNN Classifier
parameters={'n_neighbors'= 5, 'weights'='uniform', 'algorithm'='auto', 'leaf_size'=30 }
knn_random = RandomizedSearchCV(knn,parameters,n_iter=15)
knn_random.fit(X_res, y_res)
grid_parm=knn_random.best_params_
print(grid_parm)


In [0]:
#Using the parameters obtained from HyperParameterTuning in the NN 
knn_hyperParam = KNeighborsClassifier(**grid_parm)
knn_hyperParam.fit(X_res,y_res)
test_pred=pd.DataFrame(knn_hyperParam.predict(X_test),columns=["QuoteConversion_Flag"])

In [0]:
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/ResultKnnHyperParam.csv")

In [39]:
#run cross-validation on best hyperparameters, get auc score
ClfCvScore = cross_val_score(clf, X_res, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(ClfCvScore)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",ClfCvScore.mean())

=== All AUC Scores ===
[0.74554418 0.74810391 0.74611301 0.75739477 0.77379598 0.81484642
 0.818449   0.81190747 0.81661549 0.82002421]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7852794420303368


In [0]:
from sklearn.svm import LinearSVC

In [0]:
#CONSTRUCT DEFAULT SVM AND OBTAIN RESPECTIVE ACCURACY ==================
svm = LinearSVC()
svm.fit(X_res, y_res)
test_pred=pd.DataFrame(svm.predict(X_test),columns=["QuoteConversion_Flag"])
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/Result_SVM.csv")

In [0]:
#Hyperparameter tuning done for SVM Classifier
parameters={'C'=1.0, 'kernel'='sigmoid', 'degree'=3, 'gamma'='auto_deprecated', 'shrinking'=True}
sv_random = RandomizedSearchCV(sv,parameters,n_iter=15)
sv_random.fit(X_res, y_res)
grid_parm=sv_random.best_params_
print(grid_parm)

In [0]:
#Using the parameters obtained from HyperParameterTuning in the SVM
sv= LinearSVC(**grid_parm)
sv.fit(X_res,y_res)
test_pred=pd.DataFrame(sv.predict(X_test),columns=["QuoteConversion_Flag"])

In [0]:
# outputs as required on kaggle
target=pd.Series(test_pred["QuoteConversion_Flag"])
q_num=pd.Series(testData_Copy['QuoteNumber'])

#Write into a file with actual prediction
pd.concat([q_num,target],axis=1).to_csv("/gdrive/My Drive/week3/Result_SvmHyperParam.csv")

In [43]:
#run cross-validation on best hyperparameters, get auc score ======================
clf_cv_score = cross_val_score(clf, X_res, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

=== All AUC Scores ===
[0.74554418 0.74810391 0.74611301 0.75739477 0.77379598 0.81484642
 0.818449   0.81190747 0.81661549 0.82002421]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7852794420303368


In [20]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ clf,rfc,knn ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.90656285]
    fold  1:  [0.90241189]
    fold  2:  [0.90038430]
    fold  3:  [0.89967638]
    ----
    MEAN:     [0.90225885] + [0.00268006]
    FULL:     [0.90225897]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91323693]
    fold  1:  [0.91292916]
    fold  2:  [0.91039644]
    fold  3:  [0.90751416]
    ----
    MEAN:     [0.91101917] + [0.00230431]
    FULL:     [0.91101925]

model  2:     [KNeighborsClassifier]
    fold  0:  [0.70103145]
    fold  1:  [0.69444304]
    fold  2:  [0.69943366]
    fold  3:  [0.69725930]
    ----
    MEAN:     [0.69804186] + [0.00247177]
    FULL:     [0.69804189]



In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL Decision Tree===========================================
model1 = DecisionTreeClassifier()
model1 = model1.fit(S_Train, y_res)
y_pred1 = model1.predict(S_Test)

In [52]:
#Get Prediction Probability for the predicted class as a dataframe =====================================
pred_prob = pd.DataFrame(model.predict_proba(S_Test))
pred_prob.head()

,0,1
0,0.956666,0.043334
1,0.956666,0.043334
2,0.956666,0.043334
3,0.956666,0.043334
4,0.895150,0.104850


In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL Random Forest===========================================
model2 = RandomForestClassifier()
model2 = model2.fit(S_Train, y_res)
y_pred2 = model2.predict(S_Test)

In [0]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL KNN===========================================
model3 = KNeighborsClassifier()
model3 = model3.fit(S_Train, y_res)
y_pred3 = model3.predict(S_Test)

In [0]:
#Get Prediction Probability for the predicted class as a dataframe =====================================
pred_prob = pd.DataFrame(model.predict_proba(S_Test))
pred_prob.head()


In [25]:
#Write into a file with actual prediction and corresponding probability
#Get Class Prediction as a data frame with header as Prediction
pd_1=pd.DataFrame(y_pred1,columns=["QuoteConversion_Flag"]) #predict function to give training set / test set to get answers

pd_1.head() # only after hard prediction.

                
                
pd.concat([test_data[['QuoteNumber']],pd_1],axis=1).to_csv('/gdrive/My Drive/week3/Stacking_Model1.csv', index = None)

res=pd.read_csv('/gdrive/My Drive/week3/Stacking_Model1.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [28]:
res.shape

(173836, 2)

In [33]:
#Write into a file with actual prediction and corresponding probability
#Get Class Prediction as a data frame with header as Prediction
pd_2=pd.DataFrame(y_pred2,columns=["QuoteConversion_Flag"]) #predict function to give training set / test set to get answers

pd_2.head() # only after hard prediction.

                
                
pd.concat([test_data[['QuoteNumber']],pd_2],axis=1).to_csv('/gdrive/My Drive/week3/Stacking_Model2.csv', index = None)

res2=pd.read_csv('/gdrive/My Drive/week3/Stacking_Model2.csv')
res2.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [29]:
res.shape

(173836, 2)

In [35]:
#Write into a file with actual prediction and corresponding probability
#Get Class Prediction as a data frame with header as Prediction
pd_3=pd.DataFrame(y_pred3,columns=["QuoteConversion_Flag"]) #predict function to give training set / test set to get answers

pd_3.head() # only after hard prediction.

                
                
pd.concat([test_data[['QuoteNumber']],pd_3],axis=1).to_csv('/gdrive/My Drive/week3/Stacking_Model3.csv', index = None)

res3=pd.read_csv('/gdrive/My Drive/week3/Stacking_Model3.csv')
res3.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [30]:
res.shape

(173836, 2)

In [37]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm1 = SMOTE(sampling_strategy='float', ratio=0.3)
X_res1, y_res1 = sm1.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res1))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 15821})


In [39]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models1 = [ clf,rfc,knn ]
      
S_Train1, S_Test1 = stacking(models1,                   
                           X_res1, y_res1, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.89067149]
    fold  1:  [0.88366394]
    fold  2:  [0.89194235]
    fold  3:  [0.88529086]
    ----
    MEAN:     [0.88789216] + [0.00349190]
    FULL:     [0.88789218]

model  1:     [RandomForestClassifier]
    fold  0:  [0.90035587]
    fold  1:  [0.89177363]
    fold  2:  [0.89824377]
    fold  3:  [0.90086936]
    ----
    MEAN:     [0.89781066] + [0.00362171]
    FULL:     [0.89781064]

model  2:     [KNeighborsClassifier]
    fold  0:  [0.73787994]
    fold  1:  [0.73838973]
    fold  2:  [0.73936636]
    fold  3:  [0.74006651]
    ----
    MEAN:     [0.73892564] + [0.00084802]
    FULL:     [0.73892560]



In [38]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm2 = SMOTE(sampling_strategy='float', ratio=0.7)
X_res2, y_res2 = sm2.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res2))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 36916})


In [40]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models2 = [ clf,rfc,knn ]
      
S_Train2, S_Test2 = stacking(models2,                   
                           X_res2, y_res2, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.90969930]
    fold  1:  [0.90903007]
    fold  2:  [0.91085531]
    fold  3:  [0.91085531]
    ----
    MEAN:     [0.91010999] + [0.00078197]
    FULL:     [0.91010998]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91946997]
    fold  1:  [0.92339609]
    fold  2:  [0.92392808]
    fold  3:  [0.91991255]
    ----
    MEAN:     [0.92167667] + [0.00200043]
    FULL:     [0.92167667]

model  2:     [KNeighborsClassifier]
    fold  0:  [0.69759971]
    fold  1:  [0.70215044]
    fold  2:  [0.70619730]
    fold  3:  [0.69722929]
    ----
    MEAN:     [0.70079418] + [0.00367240]
    FULL:     [0.70079416]

